In [1]:
# Getting the current location
{:ok, home} = File.cwd

# Moving to project root inside the container
File.cd("/work")

# Requiring project files
Path.wildcard('/work/lib/**/*.ex')
|> Enum.map(fn (file) -> Code.require_file(file) end)

# Moving back to the previous location 
File.cd(home)

:ok

:ok

In [2]:
alias ExLearn.Matrix
alias ExLearn.NeuralNetwork, as: NN

structure_parameters = %{
  layers: %{
    input:   %{size: 2},
    hidden: [%{activity: :logistic, name: "First Hidden", size: 4}],
    output:  %{activity: :softmax,  name: "Output",       size: 2}
  },
  objective: :negative_log_likelihood
}

network = NN.create(structure_parameters)

initialization_parameters = %{
  distribution: :normal,
  deviation:    1,
  mean:         0,
  modifier:     fn(value, inputs, _outputs) -> value / :math.sqrt(inputs) end
}

NN.initialize(initialization_parameters, network)

training_data = [
  {Matrix.new(1, 2, [[0, 0]]), Matrix.new(1, 2, [[1, 0]])},
  {Matrix.new(1, 2, [[0, 1]]), Matrix.new(1, 2, [[0, 1]])},
  {Matrix.new(1, 2, [[1, 0]]), Matrix.new(1, 2, [[0, 1]])},
  {Matrix.new(1, 2, [[1, 1]]), Matrix.new(1, 2, [[1, 0]])}
]

learning_parameters = %{
  training: %{
    batch_size:     2,
    data:           training_data,
    data_size:      4,
    epochs:         600,
    learning_rate:  0.4,
    regularization: :none
  },
  workers: 2
}

NN.train(learning_parameters, network) |> Task.await(:infinity)

ask_data = [
  Matrix.new(1, 2, [[0, 0]]),
  Matrix.new(1, 2, [[0, 1]]),
  Matrix.new(1, 2, [[1, 0]]),
  Matrix.new(1, 2, [[1, 1]])
]

NN.ask(ask_data, network)
|> Task.await(:infinity)
|> Enum.map(fn(element) -> Matrix.inspect(element) end)

:ok

Rows: 1 Columns: 2
0.9813412427902222 0.01865876466035843
Rows: 1 Columns: 2
0.02071048691868782 0.9792895317077637
Rows: 1 Columns: 2
0.02051500789821148 0.9794849753379822
Rows: 1 Columns: 2
0.9729456901550293 0.027054324746131897


:ok